In [1]:
from __future__ import print_function
from scipy.io import loadmat
import numpy as np
import tensorflow as tf

/usr/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
tf.logging.set_verbosity(tf.logging.INFO)

In [3]:
SVHN_LOCATION = "/mnt/Data/SVHN/"
BATCH_SIZE = 32
EPOCHS = 20

In [4]:
def preprocessing(img, lbl):
    gray_img = tf.image.rgb_to_grayscale(img)
    norm_img = tf.image.per_image_standardization(gray_img)
    
    tens_location = tf.equal(lbl, 10 * tf.ones(lbl.shape, dtype=tf.uint8))
    tens_with_zeroes = tf.where(tens_location, tf.zeros(lbl.shape, dtype=tf.uint8),
                               lbl)
    
    return gray_img, tens_with_zeroes

In [5]:
def train_input_fn(train_mat, extra_mat):
    train_dataset = tf.data.Dataset.from_tensor_slices((np.moveaxis(train_mat['X'],
        3, 0), train_mat['y'].flatten()))
    extra_dataset = tf.data.Dataset.from_tensor_slices((np.moveaxis(extra_mat['X'],
        3, 0), extra_mat['y'].flatten()))

    #train_dataset = train_dataset.map(
    train_dataset = train_dataset.concatenate(extra_dataset).map(
        preprocessing).shuffle(10000).batch(
        BATCH_SIZE).repeat(EPOCHS - 1)

    return train_dataset

In [6]:
test_input_fn = lambda test_mat: tf.data.Dataset.from_tensor_slices((np.moveaxis(
    test_mat['X'], 3, 0), test_mat['y'].flatten())).map(preprocessing).batch(
    BATCH_SIZE)

In [7]:
def resnet_layer(inputs, filters=16, kernel_size=3, strides=1, activation=tf.nn.relu,
                 batch_normalization=True):
    # Function from keras's example for Cifar10:
    # https://github.com/keras-team/keras/blob/master/examples/cifar10_resnet.py
    x = tf.layers.conv2d(inputs=inputs, filters=filters, kernel_size=kernel_size,
                         strides=strides, padding='same')
    if batch_normalization:
        x = tf.layers.batch_normalization(x)
    if activation is not None:
        x = activation(x)
    return x

In [14]:
def resnet_model(features, labels, mode):
    # Function from keras's example for Cifar10:
    # https://github.com/keras-team/keras/blob/master/examples/cifar10_resnet.py
    filters = 16
    num_res_blocks = 9
    
    features = tf.cast(features, dtype=tf.float32)
    x = resnet_layer(inputs=features)
    for stack in range(3):
        for res_block in range(num_res_blocks):
            strides = 1
            if stack > 0 and res_block == 0:
                strides = 2
            y = resnet_layer(inputs=x, filters=filters, strides=strides)
            y = resnet_layer(inputs=y, filters=filters, activation=None)
            if stack > 0 and res_block == 0:
                x = resnet_layer(inputs=x, filters=filters, kernel_size=1,
                                 strides=strides, activation=None,
                                 batch_normalization=False)
            x = tf.nn.relu(x + y)
        filters *= 2
    
    x = tf.layers.average_pooling2d(x, pool_size=8, strides=8)
    x = tf.layers.flatten(x)
    
    #x = tf.layers.dense(inputs=x, units=10)
    w = tf.Variable(tf.truncated_normal((int(x.shape[1]), 10)))
    alpha = tf.Variable(tf.truncated_normal((int(x.shape[1]), 10)))
    hebb = tf.Variable(tf.zeros((int(x.shape[1]), 10)), trainable=False)
    eta = tf.Variable(tf.truncated_normal([1,]))
    
    b = tf.Variable(tf.truncated_normal((10,)))
    y = tf.matmul(x, w + tf.multiply(alpha, hebb)) + b
    #hebb = eta * tf.matmul(tf.transpose(x), y) + (1 - eta) * hebb
    hebb = eta * tf.reduce_mean(tf.matmul(tf.expand_dims(x, axis=-1),
        tf.expand_dims(y, axis=1)), axis=0) + (1 - eta) * hebb
    x = y
    
    classes = tf.argmax(input=x, axis=1)
    correct_prediction = tf.equal(tf.cast(classes, tf.uint8), labels)
    acc = tf.reduce_mean(tf.cast(correct_prediction, tf.float32)) * 100
    acc = tf.identity(acc, name='accuracy_tensor')
    
    predictions = {'classes': classes, 'accuracy': acc}
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    
    onehot_labels = tf.one_hot(tf.cast(labels, tf.int32), 10)
    loss = tf.nn.softmax_cross_entropy_with_logits_v2(labels=onehot_labels, logits=x)
    loss = tf.reduce_mean(loss)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.contrib.estimator.TowerOptimizer(tf.train.AdamOptimizer(1e-4))
        train_op = optimizer.minimize(loss=loss,
                                      global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    eval_metric_ops = {'accuracy': tf.metrics.accuracy(labels=labels,
                                                       predictions=classes)}
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss,
                                      eval_metric_ops=eval_metric_ops)


In [9]:
print("Loading training data...")
train_mat = loadmat(SVHN_LOCATION + "train_32x32.mat")
extra_mat = loadmat(SVHN_LOCATION + "extra_32x32.mat")
#extra_mat = train_mat
print("Loading testing data...")
test_mat = loadmat(SVHN_LOCATION + "test_32x32.mat")
print("All data loaded")

Loading training data...
Loading testing data...
All data loaded


In [10]:
tensors_to_log = {'accuracy': 'accuracy_tensor'}
logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=100)

In [17]:
model = tf.estimator.Estimator(model_fn=tf.contrib.estimator.replicate_model_fn(
    resnet_model), model_dir='./')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/mnt/Data/tmp/tf', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fbc3d76fe10>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [18]:
model.train(input_fn=lambda:train_input_fn(train_mat, extra_mat), hooks=[logging_hook])

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /mnt/Data/tmp/tf/model.ckpt.
INFO:tensorflow:accuracy = 12.5
INFO:tensorflow:loss = 7673.955, step = 0
INFO:tensorflow:global_step/sec: 3.3943
INFO:tensorflow:accuracy = 21.875 (29.462 sec)
INFO:tensorflow:loss = 10.468193, step = 100 (29.462 sec)
INFO:tensorflow:global_step/sec: 2.84682
INFO:tensorflow:accuracy = 15.625 (35.127 sec)
INFO:tensorflow:loss = 6.839806, step = 200 (35.127 sec)
INFO:tensorflow:global_step/sec: 2.84626
INFO:tensorflow:accuracy = 15.625 (35.134 sec)
INFO:tensorflow:loss = 6.0361753, step = 300 (35.134 sec)
INFO:tensorflow:global_step/sec: 2.8467
INFO:tensorflow:accuracy = 12.5 (35.128 sec)
INFO:tensorflow:loss = 3.4147902, step = 400 (35.128 sec)
INFO:tensorflow:global_step/sec: 2.

INFO:tensorflow:global_step/sec: 2.46631
INFO:tensorflow:accuracy = 71.875 (40.546 sec)
INFO:tensorflow:loss = 1.2021731, step = 5300 (40.546 sec)
INFO:tensorflow:global_step/sec: 2.46752
INFO:tensorflow:accuracy = 65.625 (40.526 sec)
INFO:tensorflow:loss = 0.9257712, step = 5400 (40.526 sec)
INFO:tensorflow:global_step/sec: 2.46278
INFO:tensorflow:accuracy = 71.875 (40.606 sec)
INFO:tensorflow:loss = 0.915647, step = 5500 (40.606 sec)
INFO:tensorflow:global_step/sec: 2.47516
INFO:tensorflow:accuracy = 71.875 (40.400 sec)
INFO:tensorflow:loss = 0.84897923, step = 5600 (40.400 sec)
INFO:tensorflow:global_step/sec: 2.47371
INFO:tensorflow:accuracy = 75.0 (40.425 sec)
INFO:tensorflow:loss = 0.6449583, step = 5700 (40.426 sec)
INFO:tensorflow:Saving checkpoints for 5756 into /mnt/Data/tmp/tf/model.ckpt.
INFO:tensorflow:global_step/sec: 2.44632
INFO:tensorflow:accuracy = 87.5 (40.878 sec)
INFO:tensorflow:loss = 0.47500032, step = 5800 (40.878 sec)
INFO:tensorflow:global_step/sec: 2.47843
IN

INFO:tensorflow:global_step/sec: 2.5247
INFO:tensorflow:accuracy = 71.875 (39.608 sec)
INFO:tensorflow:loss = 0.7505462, step = 10700 (39.608 sec)
INFO:tensorflow:global_step/sec: 2.53208
INFO:tensorflow:accuracy = 96.875 (39.493 sec)
INFO:tensorflow:loss = 0.19875388, step = 10800 (39.493 sec)
INFO:tensorflow:global_step/sec: 2.53714
INFO:tensorflow:accuracy = 87.5 (39.414 sec)
INFO:tensorflow:loss = 0.38597015, step = 10900 (39.415 sec)
INFO:tensorflow:global_step/sec: 2.53389
INFO:tensorflow:accuracy = 78.125 (39.465 sec)
INFO:tensorflow:loss = 0.44758558, step = 11000 (39.465 sec)
INFO:tensorflow:global_step/sec: 2.52786
INFO:tensorflow:accuracy = 81.25 (39.559 sec)
INFO:tensorflow:loss = 0.4454156, step = 11100 (39.559 sec)
INFO:tensorflow:global_step/sec: 2.5284
INFO:tensorflow:accuracy = 84.375 (39.551 sec)
INFO:tensorflow:loss = 0.5840401, step = 11200 (39.551 sec)
INFO:tensorflow:global_step/sec: 2.5285
INFO:tensorflow:accuracy = 90.625 (39.549 sec)
INFO:tensorflow:loss = 0.24

INFO:tensorflow:global_step/sec: 2.59271
INFO:tensorflow:accuracy = 87.5 (38.569 sec)
INFO:tensorflow:loss = 0.80612016, step = 16100 (38.569 sec)
INFO:tensorflow:global_step/sec: 2.57005
INFO:tensorflow:accuracy = 96.875 (38.910 sec)
INFO:tensorflow:loss = 0.13323632, step = 16200 (38.910 sec)
INFO:tensorflow:global_step/sec: 2.56943
INFO:tensorflow:accuracy = 84.375 (38.919 sec)
INFO:tensorflow:loss = 0.499156, step = 16300 (38.918 sec)
INFO:tensorflow:Saving checkpoints for 16383 into /mnt/Data/tmp/tf/model.ckpt.
INFO:tensorflow:global_step/sec: 2.49108
INFO:tensorflow:accuracy = 93.75 (40.143 sec)
INFO:tensorflow:loss = 0.3538286, step = 16400 (40.143 sec)
INFO:tensorflow:global_step/sec: 2.61013
INFO:tensorflow:accuracy = 90.625 (38.312 sec)
INFO:tensorflow:loss = 0.2523099, step = 16500 (38.312 sec)
INFO:tensorflow:global_step/sec: 2.57082
INFO:tensorflow:accuracy = 90.625 (38.898 sec)
INFO:tensorflow:loss = 0.3430038, step = 16600 (38.898 sec)
INFO:tensorflow:global_step/sec: 2.

INFO:tensorflow:accuracy = 87.5 (43.654 sec)
INFO:tensorflow:loss = 0.32085657, step = 21400 (43.653 sec)
INFO:tensorflow:global_step/sec: 2.35656
INFO:tensorflow:accuracy = 90.625 (42.436 sec)
INFO:tensorflow:loss = 0.33577222, step = 21500 (42.436 sec)
INFO:tensorflow:global_step/sec: 2.34989
INFO:tensorflow:accuracy = 93.75 (42.554 sec)
INFO:tensorflow:loss = 0.3075561, step = 21600 (42.555 sec)
INFO:tensorflow:global_step/sec: 2.34104
INFO:tensorflow:accuracy = 93.75 (42.716 sec)
INFO:tensorflow:loss = 0.105011016, step = 21700 (42.715 sec)
INFO:tensorflow:global_step/sec: 2.26998
INFO:tensorflow:accuracy = 96.875 (44.053 sec)
INFO:tensorflow:loss = 0.0579633, step = 21800 (44.053 sec)
INFO:tensorflow:global_step/sec: 2.32698
INFO:tensorflow:accuracy = 93.75 (42.976 sec)
INFO:tensorflow:loss = 0.18412152, step = 21900 (42.976 sec)
INFO:tensorflow:global_step/sec: 2.30258
INFO:tensorflow:accuracy = 90.625 (43.428 sec)
INFO:tensorflow:loss = 0.45814255, step = 22000 (43.427 sec)
INFO

INFO:tensorflow:loss = 0.09228484, step = 26700 (50.013 sec)
INFO:tensorflow:global_step/sec: 1.97891
INFO:tensorflow:accuracy = 93.75 (50.533 sec)
INFO:tensorflow:loss = 0.22538209, step = 26800 (50.532 sec)
INFO:tensorflow:global_step/sec: 1.97045
INFO:tensorflow:accuracy = 90.625 (50.749 sec)
INFO:tensorflow:loss = 0.31173784, step = 26900 (50.749 sec)
INFO:tensorflow:global_step/sec: 1.97024
INFO:tensorflow:accuracy = 96.875 (50.761 sec)
INFO:tensorflow:loss = 0.078626856, step = 27000 (50.761 sec)
INFO:tensorflow:global_step/sec: 1.97131
INFO:tensorflow:accuracy = 96.875 (50.722 sec)
INFO:tensorflow:loss = 0.21351105, step = 27100 (50.723 sec)
INFO:tensorflow:global_step/sec: 1.97305
INFO:tensorflow:accuracy = 96.875 (50.683 sec)
INFO:tensorflow:loss = 0.12061903, step = 27200 (50.682 sec)
INFO:tensorflow:global_step/sec: 1.96847
INFO:tensorflow:accuracy = 100.0 (50.801 sec)
INFO:tensorflow:loss = 0.03517104, step = 27300 (50.801 sec)
INFO:tensorflow:global_step/sec: 1.97005
INFO:

INFO:tensorflow:global_step/sec: 2.4837
INFO:tensorflow:accuracy = 100.0 (40.263 sec)
INFO:tensorflow:loss = 0.0150008, step = 32100 (40.263 sec)
INFO:tensorflow:global_step/sec: 2.41272
INFO:tensorflow:accuracy = 100.0 (41.447 sec)
INFO:tensorflow:loss = 0.025055371, step = 32200 (41.448 sec)
INFO:tensorflow:global_step/sec: 2.4176
INFO:tensorflow:accuracy = 96.875 (41.363 sec)
INFO:tensorflow:loss = 0.14255321, step = 32300 (41.361 sec)
INFO:tensorflow:global_step/sec: 2.40141
INFO:tensorflow:accuracy = 96.875 (41.642 sec)
INFO:tensorflow:loss = 0.047501583, step = 32400 (41.642 sec)
INFO:tensorflow:global_step/sec: 2.47379
INFO:tensorflow:accuracy = 90.625 (40.424 sec)
INFO:tensorflow:loss = 0.20272627, step = 32500 (40.424 sec)
INFO:tensorflow:global_step/sec: 2.41326
INFO:tensorflow:accuracy = 87.5 (41.438 sec)
INFO:tensorflow:loss = 0.3242916, step = 32600 (41.438 sec)
INFO:tensorflow:global_step/sec: 2.37765
INFO:tensorflow:accuracy = 96.875 (42.058 sec)
INFO:tensorflow:loss = 0

ResourceExhaustedError: /mnt/Data/tmp/tf/model.ckpt-36770_temp_23204ee4e1a74a18808a12f148fec2d1/part-00000-of-00001.data-00000-of-00001.tempstate16596747864253421487; No space left on device
	 [[Node: save/SaveV2 = SaveV2[dtypes=[DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, ..., DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_INT64], _device="/job:localhost/replica:0/task:0/device:CPU:0"](save/ShardedFilename, save/SaveV2/tensor_names, save/SaveV2/shape_and_slices, Variable/_3187, Variable/Adam/_3189, Variable/Adam_1/_3191, Variable_1/_3193, Variable_1/Adam/_3195, Variable_1/Adam_1/_3197, Variable_2/_3199, Variable_3/_3201, Variable_4/_3203, Variable_4/Adam/_3205, Variable_4/Adam_1/_3207, batch_normalization/beta/_3209, batch_normalization/beta/Adam/_3211, batch_normalization/beta/Adam_1/_3213, batch_normalization/gamma/_3215, batch_normalization/gamma/Adam/_3217, batch_normalization/gamma/Adam_1/_3219, batch_normalization/moving_mean/_3221, batch_normalization/moving_variance/_3223, b

In [13]:
print(model.evaluate(input_fn=lambda:test_input_fn(test_mat)))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-06-01-09:51:23
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./model.ckpt-67447
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-06-01-09:51:52
INFO:tensorflow:Saving dict for global step 67447: accuracy = 0.93362015, global_step = 67447, loss = 0.41024035
{'accuracy': 0.93362015, 'loss': 0.41024035, 'global_step': 67447}
